### **Preparação do ambiente para executar Spark no Google Colab**

In [0]:
# Importar o FindSpark para localizar a instalação do spark e iniciar uma sessão
import findspark
import pyspark.sql.functions as f
from pyspark.sql import SparkSession
from pyspark.sql.types import DecimalType
findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
dfAug = spark.read\
    .format("com.databricks.spark.csv")\
    .option("header", "false") \
    .option("delimiter", " ") \
    .option("inferSchema", "true") \
    .load("gdrive/My Drive/Desafio Técnico - Semantix/NASA_access_log_Aug95")


In [0]:
dfJul = spark.read\
    .format("com.databricks.spark.csv")\
    .option("header", "false") \
    .option("delimiter", " ") \
    .option("inferSchema", "true") \
    .load("gdrive/My Drive/Desafio Técnico - Semantix/NASA_access_log_Jul95")

In [0]:
df = dfJul.union(dfAug)

In [9]:
df.show(7)

+--------------------+---+---+--------------------+------+--------------------+---+----+
|                 _c0|_c1|_c2|                 _c3|   _c4|                 _c5|_c6| _c7|
+--------------------+---+---+--------------------+------+--------------------+---+----+
|        199.72.81.55|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /history/apol...|200|6245|
|unicomp6.unicomp.net|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200|3985|
|      199.120.110.21|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/miss...|200|4085|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|304|   0|
|      199.120.110.21|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/miss...|200|4179|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /images/NASA-...|304|   0|
|  burger.letters.com|  -|  -|[01/Jul/1995:00:0...|-0400]|GET /shuttle/coun...|200|   0|
+--------------------+---+---+--------------------+------+--------------------+---+----+
only showing top 7 ro

In [0]:
split_col = f.split(df['_c3'], ':')
df = df.withColumn('_c3', split_col.getItem(0))

In [0]:
df = df.withColumn('_c3', df['_c3'].substr(2, 11))

In [0]:
df = df.selectExpr('_c0 as host', '_c3 as date', '_c6 as code', '_c7 as size')

In [13]:
df.show(7)

+--------------------+-----------+----+----+
|                host|       date|code|size|
+--------------------+-----------+----+----+
|        199.72.81.55|01/Jul/1995| 200|6245|
|unicomp6.unicomp.net|01/Jul/1995| 200|3985|
|      199.120.110.21|01/Jul/1995| 200|4085|
|  burger.letters.com|01/Jul/1995| 304|   0|
|      199.120.110.21|01/Jul/1995| 200|4179|
|  burger.letters.com|01/Jul/1995| 304|   0|
|  burger.letters.com|01/Jul/1995| 200|   0|
+--------------------+-----------+----+----+
only showing top 7 rows



## **1. Número de hosts únicos.**

In [14]:
df.select('host').distinct().count()

137979

## **2. O total de erros 404**

In [15]:
df.where(df['code'] == "404").count()

20871

## **3. Os 5 URLs que mais causaram erro 404.**

In [16]:
df.where(df['code'] == "404").groupBy("host").count().orderBy("count", ascending=False).show(5)

+--------------------+-----+
|                host|count|
+--------------------+-----+
|hoohoo.ncsa.uiuc.edu|  251|
|piweba3y.prodigy.com|  156|
|jbiagioni.npt.nuw...|  132|
|piweba1y.prodigy.com|  114|
|www-d4.proxy.aol.com|   91|
+--------------------+-----+
only showing top 5 rows



## **4. Quantidade de erros 404 por dia.**

In [17]:
df.where(df['code'] == "404").groupBy("date").count().orderBy("date", ascending=True).show()

+-----------+-----+
|       date|count|
+-----------+-----+
|01/Aug/1995|  243|
|01/Jul/1995|  316|
|02/Jul/1995|  291|
|03/Aug/1995|  303|
|03/Jul/1995|  470|
|04/Aug/1995|  346|
|04/Jul/1995|  359|
|05/Aug/1995|  236|
|05/Jul/1995|  497|
|06/Aug/1995|  373|
|06/Jul/1995|  640|
|07/Aug/1995|  537|
|07/Jul/1995|  569|
|08/Aug/1995|  390|
|08/Jul/1995|  302|
|09/Aug/1995|  279|
|09/Jul/1995|  348|
|10/Aug/1995|  306|
|10/Jul/1995|  398|
|11/Aug/1995|  263|
+-----------+-----+
only showing top 20 rows



## **5. O total de bytes retornados.**

In [18]:
total = df.groupBy().agg(f.sum('size'))
total.withColumn('sum(size)', total['sum(size)'].cast(DecimalType(18, 0))).show()

+-----------+
|  sum(size)|
+-----------+
|65524319796|
+-----------+

